<h1> Model Training - Multilayer Perceptron</h1>

In [301]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import metrics
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.compose import TransformedTargetRegressor

In [302]:
df = pd.read_csv('imoveis.csv')

In [342]:
df.head()

,local,preco,metragem,quartos,garagens,banheiros,bairro
0,"Rua Maria Thereza Gonçalves, Umbara",399900,106,3.0,2.0,2,76
1,"Travessa Paulo Ribeiro, Sitio Cercado",765190,319,5.0,3.0,4,70
2,"Uberaba, Curitiba",2350000,500,5.0,7.0,5,75
3,"Rua Ângelo Greca, Atuba",737000,144,4.0,NaN,4,7
4,"Rua João Fonseca Mercer, Atuba",1140000,276,5.0,3.0,5,7


In [304]:
x = df.drop("preco", axis=1).drop("local", axis=1)
y = df["preco"]

In [306]:
scaler = MinMaxScaler()
scaler_preco = MinMaxScaler()

features = [['metragem', 'quartos', 'garagens', 'banheiros', 'bairro']]
for feature in features:
    x[feature] = scaler.fit_transform(x[feature])

y = scaler_preco.fit_transform(y.values.reshape(-1,1))

In [307]:
x.fillna(0, inplace=True)
y_credit =y
x_credit = x.to_numpy()

In [314]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x_credit, y_credit, test_size = 0.2, random_state = 0)

In [315]:
rede_neural_credit = MLPRegressor(random_state=10, max_iter=1500, early_stopping=True, validation_fraction=0.2)
rede_neural_credit.fit(x_treinamento, y_treinamento)


c:\Users\dudun\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(early_stopping=True, max_iter=1500, random_state=10,
             validation_fraction=0.2)

In [316]:
previsoes = rede_neural_credit.predict(x_teste)

In [317]:
print("MAE:", metrics.mean_absolute_error(y_teste, previsoes))
print("R2:", metrics.r2_score(y_teste, previsoes))
print("R2 Log:", metrics.mean_squared_log_error(y_teste, previsoes))

MAE: 0.013748916414163224
R2: 0.4630140052087355
R2 Log: 0.0005216351424249668


In [318]:
scaler_preco.inverse_transform(previsoes.reshape(-1, 1))

array([[ 750657.81292769],
       [ 915721.26116992],
       [ 888212.93047651],
       ...,
       [ 753836.62609319],
       [1763441.94221547],
       [ 894186.0648521 ]])

In [319]:
scaler_preco.inverse_transform(y_teste.reshape(-1, 1))

array([[ 780000.],
       [1400000.],
       [ 730000.],
       ...,
       [1100000.],
       [1909115.],
       [ 956500.]])

In [346]:
valores = scaler.fit_transform([[106,3.0,2.0,2,76]])
casa = rede_neural_credit.predict(valores)

In [347]:
scaler_preco.inverse_transform([casa])

array([[-36516.40288162]])